In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import csv
import re
import sys
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt # Library for plots
from matplotlib.pyplot import figure
import python_utils 
import sklearn.naive_bayes
import pdb
from nltk.corpus import stopwords, twitter_samples
import nltk #Natural Language Toolkit-sembolik ve istatistiksel doğal dil işleme için bir kütüphaneler ve programlar paketidir.
import string
from nltk.tokenize import TweetTokenizer #cümleyi kelimelere bölmeye yarar.
from os import getcwd
from nltk.stem import PorterStemmer #Kök oluşturma, bir kök / temel sözcüğün morfolojik varyantlarını üretir.

In [ ]:
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
import string

#PROCESS DATA
def process_tweet(tweet):
  
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
            word not in string.punctuation):  # remove punctuation
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean


def test_lookup(func):
    freqs = {('sad', 0): 4,
             ('happy', 1): 12,
             ('oppressed', 0): 7}
    word = 'happy'
    label = 1
    if func(freqs, word, label) == 12:
        return 'SUCCESS!!'
    return 'Failed Sanity Check!'


def lookup(freqs, word, label):
  
       #freqs: Her bir pair (key-value) 'in frekansıyla birlikte var olan kütüphane.
       
   
      # n: Karşılık gelen etikete -label- sahip kelimenin görünme sayısı.
    
    n = 0  # freqs.get((word, label), 0)

    pair = (word, label)
    if (pair in freqs):
        n = freqs[pair]

    return n

In [ ]:
filePath = f"{getcwd()}/...\nltk_data\corpora"
nltk.data.path.append(filePath)

In [ ]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

# Data iki parçaya bölünür,biri training(model eğitme), diğeri testing (test etme) için. (validation set)
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

# all_positive_tweets uzunluğu hakkındaki varsayımlardan sakınmak gerekiyor.
train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
test_y = np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))

In [ ]:
def count_tweets(result, tweets, ys):
    
#result: Frequency ile tweet-sentiment değer ikilisini tutan kütüphane

    for tweet, y in zip(tweets,ys):        #ys: 0/1 
        for word in process_tweet(tweet):
         
            pair = (word, y) # örneğin word=happy y=1 

           
            if pair in result:
                result[pair] += 1

           
            else:
                result[pair] = 1


    return result

In [ ]:
def train_naive_bayes(freqs, train_x, train_y):
   
    #frequency: kelime ve 0/1 değeriyle kullanılan kelimenin kullanılma sıklığı
    #train_x: Tweet listesi
    #train_y:Tweet'lere karşılık gelen 0/1 etiketlerin listesi
    
    #logprior=log(Dpos)-log(dneg)
    #loglikelihood:log(p(Wpos)/p(Wneg))
    
    loglikelihood = {}
    logprior = 0

  
    vocab = set([pair[0] for pair in freqs.keys()])
    V = len(vocab)

    
    N_pos = N_neg = V_pos = V_neg = 0
    for pair in freqs.keys():
     
        if pair[1] > 0:
            
            V_pos += 1

            N_pos += freqs[pair]

        else:
            
            V_neg += 1

           
            N_neg += freqs[pair]


    D = len(train_y)

    D_pos = (len(list(filter(lambda x: x > 0, train_y))))

    D_neg = (len(list(filter(lambda x: x <= 0, train_y))))

    logprior = np.log(D_pos) - np.log(D_neg)

  
    for word in vocab:
        # Her kelimenin negatif ve pozitif frekansları
        freq_pos = lookup(freqs,word,1)
        freq_neg = lookup(freqs,word,0)

        # Her kelimenin negatif ve pozitif olasılıkları hesaplanır.
        p_w_pos = (freq_pos + 1) / (N_pos + V)
        p_w_neg = (freq_neg + 1) / (N_neg + V)

        # Her kelimenin log likelihood eşitliği bulunur.
        loglikelihood[word] = np.log(p_w_pos/p_w_neg)

    return logprior, loglikelihood

In [ ]:
freqs = count_tweets({}, train_x, train_y)

In [ ]:
logprior, loglikelihood = train_naive_bayes(freqs, train_x, train_y)
print(logprior)
print(len(loglikelihood))

In [ ]:
def naive_bayes_predict(tweet, logprior, loglikelihood):
     
        
    #loglikelihood: Sayılarla eşleşen kelimelerin kütüphanesi.
   
    # Listedeki kelimeleri geri getirebilmek için yapılan işlem
    word_l = process_tweet(tweet)

    # Olasılık sıfıra eşitlenir önce.
    p = 0

    # logprior sayısı eklenir.
    p += logprior

    for word in word_l:

        # Kelimenin loglikelihood kütüphanesinde olup olmadığı kontrol edilir. 
        if word in loglikelihood:
            #Olasılığa o kelimenin loglikelihood değeri eklenir.
            p += loglikelihood[word]   
             #p: tweet'teki her kelimenin tüm loglikelihood'larının toplamı (sözlükte bulunursa) + logprior 

  

    return p

In [ ]:
def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
 
    
         #test_x: Tweet'lerin listesi
         #test_y: Tweetlere karşılık gelen labeller/etiketler
         
         #loglikelihood: Her sözcüğün loglikelihoods değerlerini içeren kütüphane
   
        #accuracy/kesinlik:(Doğru sınıflandırılan  # tweet sayısı) / (toplam # tweet sayısı)
   
    accuracy = 0  

   
    y_hats = []
    for tweet in test_x:
        # Tahmin > 0
        if naive_bayes_predict(tweet, logprior, loglikelihood) > 0:
            # Tahmin edilen class 1'dir.
            y_hat_i = 1
        else:
            # Aksi takdirde 0'dır.
            y_hat_i = 0

        # Tahmin edilen sınıfı y_hats listesine ekler.
        y_hats.append(y_hat_i)

    # Hata, y_hats ve test_y arasındaki farkların mutlak değerlerinin ortalamasıdır.
    error = np.mean(np.absolute(y_hats-test_y))

    # 1-error kesinliği verir.
    accuracy = 1-error

   
    return accuracy

In [ ]:
print("Naive Bayes accuracy = %0.4f" %
      (test_naive_bayes(test_x, test_y, logprior, loglikelihood)))

In [ ]:
def get_ratio(freqs, word):
    
   #freqs: Kelimeleri içeren kütüphane
   #word hesaplanan kelimenin kendisi

    pos_neg_ratio = {'positive': 0, 'negative': 0, 'ratio': 0.0}
    
    # lookup() kelimeler için pozitif sayıları bulur. (1 ile belirtilir.)
    pos_neg_ratio['positive'] = lookup(freqs,word,1)

    # lookup() kelimeler için negatif sayıları bulur. (0 ile belirtilir.)
    pos_neg_ratio['negative'] = lookup(freqs,word,0)

    # Kelime için pozitif ve negatif sayıların oranı hesaplanır.
    pos_neg_ratio['ratio'] = (pos_neg_ratio['positive'] + 1)/(pos_neg_ratio['negative'] + 1)
   
    return pos_neg_ratio #Pozitif,negatif ve oran anahtar kelimeleri içeren kütüphane

In [ ]:
def get_words_by_threshold(freqs, label, threshold):
    
    word_list = {}

    #freqs: kelime kütüphanesi
    #pos_neg_ratio: Pozitif,negatif sayılarını ve pozitif/negatif oranlarının sayılarının kütüphanesi
    # label: pozitifse 1, negatifse 0.
    # threshold/eşik değeri: Döndürülen sözlüğe bir kelime eklemek için kesme noktası olarak kullanılacaktır.
    #word_list:Kelime ve pozitif sayısı, negatif sayısı ve pozitif / negatif sayıları oranı hakkında bilgi içeren sözlük.
    #Anahtar değer ikilisi örneğin; 
       #  {'happi':
       #     {'positive': 10, 'negative': 20, 'ratio': 0.5}
       #  }
        
    for key in freqs.keys():
        word, _ = key

        # kelimenin negatif/pozitif oranını buluruz.
        pos_neg_ratio = get_ratio(freqs, word)

        # Eğer etiket 1 ise ve oran eşik değere büyük eşit veya eşitse; 
        if label == 1 and pos_neg_ratio['ratio'] >= threshold :

            # pos_neg_ratio kütüphaneye eklenir.
            word_list[word] = pos_neg_ratio

      
        # Eğer etiket 0 ise ve oran eşik değere küçük eşit veya eşitse; 
        elif label == 0 and pos_neg_ratio['ratio'] <= threshold:
            # pos_neg_ratio kütüphaneye eklenir.
            word_list[word] = pos_neg_ratio

      
    return word_list

In [ ]:
get_words_by_threshold(freqs, label=0, threshold=0.05)

In [ ]:
get_words_by_threshold(freqs, label=1, threshold=10)

In [ ]:
print('Truth Predicted Tweet')
for x, y in zip(test_x, test_y):
    y_hat = naive_bayes_predict(x, logprior, loglikelihood)
    if y != (np.sign(y_hat) > 0):
        print('%d\t%0.2f\t%s' % (y, np.sign(y_hat) > 0, ' '.join(
            process_tweet(x)).encode('ascii', 'ignore')))

In [ ]:
data = pd.read_excel (r'C:\Users\Lenovo\Kitap1.xlsx')
#df.rename(columns={0: 'Sentences', 1:'Sentiment'}, inplace=True)
#df = pd.DataFrame(data, columns= ['Sentiment']) sadece 1 column için
data['Sentiment'] = np.array([naive_bayes_predict(text, logprior, loglikelihood) for text in data['Sentences']])
data

In [ ]:
plt.figure(figsize=(17,5))
plt.plot(data.Sentiment,'o')
plt.legend(['Haber1']) #Haber1

plt.ylabel('Sentiment Değerleri') #Sentiment Values
plt.title('"TV series "Ertugrul" fuels Pakistanis’ love for Turkey"') #Title of the news

plt.grid(True)
plt.show()

In [ ]:
#Haberdeki pozitif, negatif, nötr sentimentli cümlelerin haberdeki yüzdeleri | Percentage of negative/positive/neutral sentences in the news 
ptweets=data[data.Sentiment>0]

print("Pozitif Cümlelerin Yüzdesi: {} %".format(100*len(ptweets)/len(data))) 
   
ntweets = data[data.Sentiment <0 ]
   
print("Negatif Cümlelerin Yüzdesi: {} %".format(100*len(ntweets)/len(data))) 

print("Nötr Cümlelerin Yüzdesi: {} % ".format(100*(len(data) -(len( ntweets )+len( ptweets)))/len(data))) 

In [ ]:
#Haberin toplam duygu değeri | Total sentiment value of news
result=0
for sentence in data['Sentiment'] :
    result+= sentence
result 